In [1]:
%%bash
pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 KB 26.7 MB/s eta 0:00:00


In [2]:
import csv
import re
import pandas as pd

In [3]:
# Define input and output file paths
input_file = 'scala_commands_with_MIN.txt'
output_file = 'results/features.csv'

# Open input and output files
with open(input_file, 'r') as f_input, open(output_file, 'w', newline='') as f_output:
    csv_writer = csv.writer(f_output)
    
    # Write header to CSV file
    csv_writer.writerow(['bench', 'query', '#relations', '#conditions', '#filters', '#joins', 'text'])
    
    # Read input file line by line
    for line in f_input:
        # Split each line into components
        pattern = r'(?<!\\)\"|\"(?<!\\)(?=\s+\"|$)'
        components = re.split(pattern, line)
        
        # Extract relevant information
        stats = components[3]
        number = components[5]
        query = components[1].strip()
        
        # get the number of relations
        query_upper = query.upper()
        from_index = query_upper.find("FROM")
        where_index = query_upper.find("WHERE")
        number_of_relations = query[from_index:where_index].count(",") + 1

        # get the number of conditions
        number_of_conditions = query.count("=")

        # get how many filter and join conditions
        parts = query_upper.split("WHERE")[1].split("AND")
        filter = 0
        join = 0
        for p in parts:
            if p.count(".") == 1:
                filter += 1
            else:
                join += 1
                
        # Write data to CSV file
        csv_writer.writerow([stats, number, number_of_relations, number_of_conditions, filter, join, query])

In [4]:
# Read the CSV files into Pandas DataFrames
df1 = pd.read_csv('results/features.csv')
df2 = pd.read_csv('results/POS_Scala_comparison_TO.csv')

# Merge the DataFrames based on a common column
# Replace 'common_column_name' with the actual common column name
merged_df = pd.merge(df1, df2, on=["bench", "query"], how='inner')

# Now, merged_df contains the combined data from both CSV files
merged_df[["#relations", "orig/rewr(mean)", "text"]]

,#relations,orig/rewr(mean),text
0,3,orig,"SELECT MIN(u.Id) FROM comments as c, votes as ..."
1,5,orig,"SELECT MIN(c.Id) FROM comments as c, postHisto..."
2,6,orig,"SELECT MIN(c.Id) FROM comments as c, posts as ..."
3,4,orig,"SELECT MIN(pl.Id) FROM postLinks as pl, posts ..."
4,3,orig,"SELECT MIN(v.Id) FROM votes as v, badges as b,..."
...,...,...,...
156,7,orig,SELECT MIN(n.name) AS member_in_charnamed_movi...
157,7,orig,SELECT MIN(n.name) AS member_in_charnamed_movi...
158,10,orig,SELECT MIN(t.title) AS complete_downey_ironman...
159,10,orig,SELECT MIN(t.title) AS complete_downey_ironman...


In [5]:
merged_df[["query", "#relations", "#conditions", "#filters", "#joins", "orig/rewr(mean)", "orig/rewr+rewr(mean)", "text"]].to_csv('results/features_times.csv', index=False)